<center>
<img src="https://laelgelcpublic.s3.sa-east-1.amazonaws.com/lael_50_years_narrow_white.png.no_years.400px_96dpi.png" width="300" alt="LAEL 50 years logo">
<h3>APPLIED LINGUISTICS GRADUATE PROGRAMME (LAEL)</h3>
</center>
<hr>

# Corpus Linguistics - Study 1 - Quérem

## Prerequisites

Make sure the prerequisites in [CL_LMDA_prerequisites](https://github.com/laelgelc/laelgelc/blob/main/CL_LMDA_prerequisites.ipynb) are satisfied.

## Dataset

Please download the following dataset (Right-click on the link and choose `Open link in a new tab` to download the corresponding file):
- [tweets_all2.tsv](https://pucsp-my.sharepoint.com/:u:/g/personal/ra00341729_pucsp_edu_br/Edr0Gb9w9s1CsLhz8bpGVn8BGQwPRoXeiM01wpCDWpN5qA?e=g1WZwE)

## Importing the required libraries

In [1]:
import pandas as pd
import demoji
import re
import os
from collections import Counter

## Data wrangling

### Importing the tweet raw data into a dataframe

In [2]:
df_tweets_raw_data = pd.read_csv('tweets_all2.tsv', sep='\t')

In [3]:
df_tweets_raw_data.head(5)

,file,created_at,text_uniq,conversation_id,author_id,username,tweet_url,text,text_emojified,photo_uniq_id
0,file,created_at,tweet_00000001,conversation_id,author_id,username,tweet_url,text,text_emojified,file00000001.url
1,bolsonaro201803_n_00101.json,2018-03-30 10:21:59,tweet_00000002,convo_979665031997612034,id_731971002817744896,gizelda_m,https://twitter.com/gizelda_m/status/979665031...,RT @MendesOnca: Fracasso Bolsonaro só tem públ...,RT @MendesOnca: Fracasso Bolsonaro só tem públ...,NaN
2,bolsonaro201803_n_00101.json,2018-03-30 10:21:32,tweet_00000003,convo_979664919955214337,id_14372459,unknown,unknown,"RT @pelegrini65: Após caluniar, ameaçar, incit...","RT @pelegrini65: Após caluniar, ameaçar, incit...",NaN
3,bolsonaro201803_n_00101.json,2018-03-28 15:31:51,tweet_00000004,convo_979018234736373760,id_287765295,pelegrini65,https://twitter.com/pelegrini65/status/9790182...,"Após caluniar, ameaçar, incitar as pessoas con...","Após caluniar, ameaçar, incitar as pessoas con...",NaN
4,bolsonaro201803_n_00101.json,2018-03-30 10:21:21,tweet_00000005,convo_979664873788502016,id_2551060160,PauliGVivas,https://twitter.com/PauliGVivas/status/9796648...,"RT @BohnGass: ""Nenhum apoiador de Bolsonaro ap...","RT @BohnGass: ""Nenhum apoiador de Bolsonaro ap...",NaN


In [4]:
# Dropping the first row, which contains no useful data, and resetting the index
df_tweets_raw_data = df_tweets_raw_data.drop(index=0).reset_index(drop=True)

In [5]:
# Dropping the columns 'text_emojified' and 'photo_uniq_id' which are not used in this analysis
df_tweets_raw_data = df_tweets_raw_data.drop(columns=['text_emojified', 'photo_uniq_id'])

In [6]:
df_tweets_raw_data

,file,created_at,text_uniq,conversation_id,author_id,username,tweet_url,text
0,bolsonaro201803_n_00101.json,2018-03-30 10:21:59,tweet_00000002,convo_979665031997612034,id_731971002817744896,gizelda_m,https://twitter.com/gizelda_m/status/979665031...,RT @MendesOnca: Fracasso Bolsonaro só tem públ...
1,bolsonaro201803_n_00101.json,2018-03-30 10:21:32,tweet_00000003,convo_979664919955214337,id_14372459,unknown,unknown,"RT @pelegrini65: Após caluniar, ameaçar, incit..."
2,bolsonaro201803_n_00101.json,2018-03-28 15:31:51,tweet_00000004,convo_979018234736373760,id_287765295,pelegrini65,https://twitter.com/pelegrini65/status/9790182...,"Após caluniar, ameaçar, incitar as pessoas con..."
3,bolsonaro201803_n_00101.json,2018-03-30 10:21:21,tweet_00000005,convo_979664873788502016,id_2551060160,PauliGVivas,https://twitter.com/PauliGVivas/status/9796648...,"RT @BohnGass: ""Nenhum apoiador de Bolsonaro ap..."
4,bolsonaro201803_n_00101.json,2018-03-29 21:06:45,tweet_00000006,convo_979464906327158784,id_63127680,BohnGass,https://twitter.com/BohnGass/status/9794649063...,Nenhum apoiador de Bolsonaro apareceu no local...
...,...,...,...,...,...,...,...,...
1783133,bolsonaro202304_n_00206.json,2023-04-29 19:17:53,tweet_00480328,convo_1652391775812681728,id_1062737655954317312,unknown,unknown,RT @Jakelyneloiola_: Tarcísio replica política...
1783134,bolsonaro202304_n_00206.json,2023-04-29 01:32:46,tweet_00480329,convo_1652123726853337094,id_1114237991797522434,Jakelyneloiola_,https://twitter.com/Jakelyneloiola_/status/165...,Tarcísio replica política de Bolsonaro e distr...
1783135,bolsonaro202304_n_00206.json,2023-04-29 19:17:52,tweet_00480507,convo_1652391768954945541,id_1081423351057448960,GisleideZV,https://twitter.com/GisleideZV/status/16523917...,RT @Zoro_ZV: Passando só pra lembrar à Direta ...
1783136,bolsonaro202304_n_00206.json,2023-04-29 19:17:51,tweet_00487148,convo_1652389098881941507,id_1614107228805414913,unknown,unknown,@lucianoaugusto1 @choquei fds pega a filó e en...


### Inspecting the dataset and eliminating malformed data

#### Checking if data types are consistent

In [7]:
df_tweets_raw_data.dtypes

file               object
created_at         object
text_uniq          object
conversation_id    object
author_id          object
username           object
tweet_url          object
text               object
dtype: object

#### Identifying rows that are empty in column `text`

In [8]:
print(df_tweets_raw_data['text'].isnull().sum())

24


In [9]:
df_tweets_raw_data[df_tweets_raw_data['text'].isnull()]

,file,created_at,text_uniq,conversation_id,author_id,username,tweet_url,text
18096,#Bolsonaro2018,@ultimosegundo Ahhhhhhhh.. Então tem q prender...,NaN,NaN,NaN,NaN,NaN,NaN
18097,#Bolsonaro2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19543,Alguém no PT deve ter uma paixão platônica pel...,@analisdocsPF @paulo_x Foi o Bolsonaro! {{Emoj...,NaN,NaN,NaN,NaN,NaN,NaN
19544,Alguém no PT deve ter uma paixão platônica pel...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37888,Generalizando e rotulando tem que cortar mal p...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37889,Bolsonaro racista safado!,@Tilitelly2 @VEJA @MPF_PGR Ele falou que todos...,NaN,NaN,NaN,NaN,NaN,NaN
37890,Generalizando e rotulando tem que cortar mal p...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37891,Bolsonaro racista safado!,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49732,"Oportunista, irresponsável.",@jairbolsonaro,NaN,NaN,NaN,NaN,NaN,NaN
49733,"Oportunista, irresponsável.",NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Dropping the rows that are empty in the column `text`

In [10]:
# Drop the rows whose column 'text' is NaN
df_tweets_raw_data = df_tweets_raw_data.dropna(subset=['text'])

# Reset the index
df_tweets_raw_data = df_tweets_raw_data.reset_index(drop=True)

In [11]:
print(df_tweets_raw_data['text'].isnull().sum())

0


#### Removing specific Unicode characters

The dataset may need to be cleaned of invisible Unicode characters.

##### Detecting `U+2066` and `U+2069` characters

- [U+2066](https://www.compart.com/en/unicode/U+2066)
- [U+2069](https://www.compart.com/en/unicode/U+2069)

Please refer to:
- [Python RegEx](https://www.w3schools.com/python/python_regex.asp)
- [regex101](https://regex101.com/)
- [RegExr](https://regexr.com/)

In [12]:
# Defining a function to detect specific Unicode characters
def extract_unicode_characters(df, column_name):
    unicode_chars = Counter()  # Initialize a Counter to store Unicode character counts

    for value in df[column_name]:
        if isinstance(value, str):
            # Use RegEx to find non-ASCII characters (Unicode)
#            non_ascii_chars = re.findall(r'[^\x00-\x7F]+', value)
            # Use RegEx to find specific Unicode characters - adjust the expression accordingly
            specific_unicode_chars = re.findall(r'[\u2066\u2069]', value)
            unicode_chars.update(specific_unicode_chars)

    return unicode_chars

# Inspect the dataframe for specific Unicode characters
unicode_counts = extract_unicode_characters(df_tweets_raw_data, 'text')

# Print the results
for char, count in unicode_counts.items():
    print(f'Character {char}: Count = {count}')

Character ⁦: Count = 986
Character ⁩: Count = 822


##### Removing `U+2066` and `U+2069` characters

In [13]:
# Defining a function to remove specific Unicode characters
def remove_specific_unicode(input_line):
    # Using RegEx to replace specific Unicode characters - adjust the expression accordingly
    cleaned_line = re.sub(r'[\u2066\u2069]', '', input_line)
    return cleaned_line

# Removing specific Unicode characters
df_tweets_raw_data['text'] = df_tweets_raw_data['text'].apply(remove_specific_unicode)

### Dropping duplicates

#### Retweets

Retweets bear the RegEx pattern `/\bRT @/gm` or `/\brt @/gm` at the beginning of the column `text`

In [14]:
# Creating a boolean mask for filtering - it is preceded by '~' to invert the selection
mask = ~df_tweets_raw_data['text'].str.contains(r'\bRT @|\brt @', regex=True)

# Applying the mask to overwrite the raw data dataframe with non retweeted tweets
df_tweets_raw_data = df_tweets_raw_data[mask]
df_tweets_raw_data = df_tweets_raw_data.reset_index(drop=True)

#### Duplicate tweets

The dataset was build in a way that if a certain tweet had more than one photo, one copy of the tweet was included per unique photo. Since we are concerned with analysing just the text, those duplicates should be removed. Tweets that bear the same 'tweet_url' are duplicates - we are going to keep only the first.

In [15]:
df_tweets_raw_data.drop_duplicates(subset='tweet_url', keep='first', inplace=True)
df_tweets_raw_data = df_tweets_raw_data.reset_index(drop=True)

#### @mentioned tweets

A few users @mention copies of tweets towards other specific users creating multiple copies of the same tweet - those duplicates should be removed.

In [16]:
# Create a new column 'no_mention' containing the contents of the column 'text' without any preceding @mentions
df_tweets_raw_data['no_mention'] = df_tweets_raw_data['text'].str.replace(r'@\w+\s*', '', regex=True)

# Drop duplicate rows except the first occurrence based on 'no_mention'
df_tweets_raw_data.drop_duplicates(subset='no_mention', keep='first', inplace=True)
df_tweets_raw_data = df_tweets_raw_data.reset_index(drop=True)

## Sampling the raw data according to filtering expressions

In [17]:
# Defining the filtering expressions
filter_words = ['arma', 'pátria', 'ladrão', 'cristão', 'comunista', 'família', 'liberdade', 'conservador', 'deus']

# Creating a boolean mask for filtering
mask = df_tweets_raw_data['text'].str.contains('|'.join(filter_words), case=False)

# Applying the mask to create 'df_tweets_filtered'
df_tweets_filtered = df_tweets_raw_data[mask]
df_tweets_filtered = df_tweets_filtered.reset_index(drop=True)

In [18]:
df_tweets_filtered

,file,created_at,text_uniq,conversation_id,author_id,username,tweet_url,text,no_mention
0,bolsonaro201803_n_00101.json,2018-03-28 15:31:51,tweet_00000004,convo_979018234736373760,id_287765295,pelegrini65,https://twitter.com/pelegrini65/status/9790182...,"Após caluniar, ameaçar, incitar as pessoas con...","Após caluniar, ameaçar, incitar as pessoas con..."
1,bolsonaro201803_n_00101.json,2018-03-30 04:12:13,tweet_00000046,convo_979571975545872384,id_16794066,BlogdoNoblat,https://twitter.com/BlogdoNoblat/status/979571...,Bolsonaro deve saber o que está fazendo. Porqu...,Bolsonaro deve saber o que está fazendo. Porqu...
2,bolsonaro201803_n_00101.json,2018-03-30 10:16:00,tweet_00000051,convo_979363111089135617,id_955901617148235776,MariaOl25529153,https://twitter.com/MariaOl25529153/status/979...,"@FlavioBolsonaro Mais um Romário na política ,...","Mais um Romário na política , que Deus ajude o..."
3,bolsonaro201803_n_00102.json,2018-03-28 18:29:28,tweet_00000100,convo_979062935103459331,id_44449830,lucianagenro,https://twitter.com/lucianagenro/status/979062...,A esquerda não tem conseguido comunicar suas p...,A esquerda não tem conseguido comunicar suas p...
4,bolsonaro201803_n_00102.json,2018-03-30 07:02:08,tweet_00000111,convo_979494088041955328,id_2832082022,AurystellaS,https://twitter.com/AurystellaS/status/9794940...,@BlogdoNoblat Vc sabe informar quantas vezes B...,Vc sabe informar quantas vezes Bolsonaro respi...
...,...,...,...,...,...,...,...,...,...
17174,bolsonaro202304_n_00205.json,2023-04-29 19:20:14,tweet_00487107,convo_1652060609674969091,id_1608481812707250176,diniz_benedicto,https://twitter.com/diniz_benedicto/status/165...,@RogerioCorreia_ Rogério Correia. Estou lendo ...,"Rogério Correia. Estou lendo ""Os Demônios Desc..."
17175,bolsonaro202304_n_00205.json,2023-04-29 19:19:18,tweet_00487120,convo_1652359568280678401,id_1547227306913153026,LuccaSo44679209,https://twitter.com/LuccaSo44679209/status/165...,"@CiresCanisio Não, o Lula não da uma refinaria...","Não, o Lula não da uma refinaria atroca de pro..."
17176,bolsonaro202304_n_00205.json,2023-04-29 19:19:14,tweet_00487123,convo_1652308555968532481,id_1554492869825683457,Andre19lll,https://twitter.com/Andre19lll/status/16523085...,@eunaovoupararde @CarlosZarattini Os índices d...,"Os índices de desemprego, PIB, inflação e qual..."
17177,bolsonaro202304_n_00206.json,2023-04-29 19:18:59,tweet_00487127,convo_1651881289497149440,id_1585200142440882179,priscila19865,https://twitter.com/priscila19865/status/16518...,@ValS265451870 @Guthbsb @marcia_miami Tá errad...,"Tá errado,EU podendo m.a.t.a.v.a um FDP desse,..."


### Exporting the filtered data into a file for inspection

In [19]:
df_tweets_filtered.to_csv('tweets_emojified.tsv', sep='\t', index=False)

## Replacing emojis

### Demojifying the column `text`

In [20]:
# Defining a function to demojify a string
def demojify_line(input_line):
    demojified_line = demoji.replace_with_desc(input_line, sep='<em>')
    return demojified_line

df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(demojify_line)

#### Exporting the filtered data into a file for inspection

In [21]:
df_tweets_filtered.to_csv('tweets_demojified1.tsv', sep='\t', index=False)

### Separating the demojified strings with spaces

In [22]:
# Defining a function to separate the demojified strings with spaces
def preprocess_line(input_line):
    # Add a space before the first delimiter '<em>', if it is not already preceded by one
    preprocessed_line = re.sub(r'(?<! )<em>', ' <em>', input_line)
    # Add a space after the first delimiter '<em>', if it is not already followed by one
    preprocessed_line = re.sub(r'<em>(?! )', '<em> ', preprocessed_line)
    return preprocessed_line

# Separating the demojified strings with spaces
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(preprocess_line)

#### Exporting the filtered data into a file for inspection

In [23]:
df_tweets_filtered.to_csv('tweets_demojified2.tsv', sep='\t', index=False)

### Formatting the demojified strings

In [24]:
# Defining a function to format the demojified string
def format_demojified_string(input_line):
    # Defining a function to format the demojified string using RegEx
    def process_demojified_string(s):
            # Lowercase the string
            s = s.lower()
            # Replace spaces and colons followed by a space with underscores
            s = re.sub(r'(: )| ', '_', s)
            # Add the appropriate prefixes and suffixes
            s = f'EMOJI{s}e'
            return s

    # Use RegEx to find and process each demojified string
    processed_line = re.sub(r'<em>(.*?)<em>', lambda match: process_demojified_string(match.group(1)), input_line)
    return processed_line

# Formatting the demojified strings
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(format_demojified_string)

### Replacing the `pipe` character by the `-` character in the `text` column

Further on, a few columns of the dataframe are going to be exported into the file `tweets.txt` whose columns need to be delimited by the `pipe` character. Therefore, it is recommended that any occurrences of the `pipe` character in the `text` column are replaced by another character.

In [25]:
# Defining a function to replace the 'pipe' character by the '-' character
def replace_pipe_with_hyphen(input_string):
    modified_string = re.sub(r'\|', '-', input_string)
    return modified_string

# Replacing the 'pipe' character by the '-' character
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(replace_pipe_with_hyphen)


#### Exporting the filtered data into a file for inspection

In [26]:
df_tweets_filtered.to_csv('tweets_demojified3.tsv', sep='\t', index=False)

## Tokenising

Please refer to [What is tokenization in NLP?](https://www.analyticsvidhya.com/blog/2020/05/what-is-tokenization-nlp/).

In [27]:
# Defining a function to tokenise a string
def tokenise_string(input_line):
    # Replace URLs with placeholders
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\b'
    placeholder = '<URL>'  # Choose a unique placeholder
    urls = re.findall(url_pattern, input_line)
    tokenised_line = re.sub(url_pattern, placeholder, input_line)  # Replace URLs with placeholders
    
    # Replace curly quotes with straight ones
    tokenised_line = tokenised_line.replace('“', '"').replace('”', '"').replace("‘", "'").replace("’", "'")
    # Separate common punctuation marks with spaces
    tokenised_line = re.sub(r'([.\!?,"\'/()])', r' \1 ', tokenised_line)
    # Add a space before '#'
    tokenised_line = re.sub(r'(?<!\s)#', r' #', tokenised_line)  # Add a space before '#' if it is not already preceded by one
    # Reduce extra spaces by a single space
    tokenised_line = re.sub(r'\s+', ' ', tokenised_line)
    
    # Replace the placeholders with the respective URLs
    for url in urls:
        tokenised_line = tokenised_line.replace(placeholder, url, 1)
    
    return tokenised_line

# Tokenising the strings
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(tokenise_string)

## Creating the files `file_index.txt` and `tweets.txt`

### Creating column `text_id`

In [28]:
df_tweets_filtered['text_id'] = 't' + df_tweets_filtered.index.astype(str).str.zfill(6)

### Creating column `conversation`

In [29]:
df_tweets_filtered['conversation'] = 'v:' + df_tweets_filtered['author_id'].str.replace('id_', '')

### Creating column `date`

In [30]:
# Convert 'created_at' to datetime format
df_tweets_filtered['created_at'] = pd.to_datetime(df_tweets_filtered['created_at'])

# Extract the date part (without time) into a new column 'date'
df_tweets_filtered['date'] = df_tweets_filtered['created_at'].dt.date

# Add the prefix 'd:' to the 'date' values
df_tweets_filtered['date'] = 'd:' + df_tweets_filtered['date'].astype(str)

### Creating column `text_url`

In [31]:
df_tweets_filtered['text_url'] = 'url:' + df_tweets_filtered['tweet_url']

### Creating column `user`

In [32]:
df_tweets_filtered['user'] = 'u:' + df_tweets_filtered['username']

### Creating column `content`

In [33]:
df_tweets_filtered['content'] = 'c:' + df_tweets_filtered['text']

### Reordering the created columns

Please refer to:
- [Python - List Comprehension 1](https://www.w3schools.com/python/python_lists_comprehension.asp)
- [Python - List Comprehension 2](https://treyhunner.com/2015/12/python-list-comprehensions-now-in-color/)

In [34]:
# Reorder the columns (we use list comprehension to create a list of all columns except 'text_id', 'variable', 'date' and 'text_url')
df_tweets_filtered = df_tweets_filtered[['text_id', 'conversation', 'date', 'text_url', 'user', 'content'] + [col for col in df_tweets_filtered.columns if col not in ['text_id', 'conversation', 'date', 'text_url', 'user', 'content']]]

In [35]:
df_tweets_filtered

,text_id,conversation,date,text_url,user,content,file,created_at,text_uniq,conversation_id,author_id,username,tweet_url,text,no_mention
0,t000000,v:287765295,d:2018-03-28,url:https://twitter.com/pelegrini65/status/979...,u:pelegrini65,"c:Após caluniar , ameaçar , incitar as pessoas...",bolsonaro201803_n_00101.json,2018-03-28 15:31:51,tweet_00000004,convo_979018234736373760,id_287765295,pelegrini65,https://twitter.com/pelegrini65/status/9790182...,"Após caluniar , ameaçar , incitar as pessoas c...","Após caluniar, ameaçar, incitar as pessoas con..."
1,t000001,v:16794066,d:2018-03-30,url:https://twitter.com/BlogdoNoblat/status/97...,u:BlogdoNoblat,c:Bolsonaro deve saber o que está fazendo . Po...,bolsonaro201803_n_00101.json,2018-03-30 04:12:13,tweet_00000046,convo_979571975545872384,id_16794066,BlogdoNoblat,https://twitter.com/BlogdoNoblat/status/979571...,Bolsonaro deve saber o que está fazendo . Porq...,Bolsonaro deve saber o que está fazendo. Porqu...
2,t000002,v:955901617148235776,d:2018-03-30,url:https://twitter.com/MariaOl25529153/status...,u:MariaOl25529153,c:@FlavioBolsonaro Mais um Romário na política...,bolsonaro201803_n_00101.json,2018-03-30 10:16:00,tweet_00000051,convo_979363111089135617,id_955901617148235776,MariaOl25529153,https://twitter.com/MariaOl25529153/status/979...,"@FlavioBolsonaro Mais um Romário na política ,...","Mais um Romário na política , que Deus ajude o..."
3,t000003,v:44449830,d:2018-03-28,url:https://twitter.com/lucianagenro/status/97...,u:lucianagenro,c:A esquerda não tem conseguido comunicar suas...,bolsonaro201803_n_00102.json,2018-03-28 18:29:28,tweet_00000100,convo_979062935103459331,id_44449830,lucianagenro,https://twitter.com/lucianagenro/status/979062...,A esquerda não tem conseguido comunicar suas p...,A esquerda não tem conseguido comunicar suas p...
4,t000004,v:2832082022,d:2018-03-30,url:https://twitter.com/AurystellaS/status/979...,u:AurystellaS,c:@BlogdoNoblat Vc sabe informar quantas vezes...,bolsonaro201803_n_00102.json,2018-03-30 07:02:08,tweet_00000111,convo_979494088041955328,id_2832082022,AurystellaS,https://twitter.com/AurystellaS/status/9794940...,@BlogdoNoblat Vc sabe informar quantas vezes B...,Vc sabe informar quantas vezes Bolsonaro respi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17174,t017174,v:1608481812707250176,d:2023-04-29,url:https://twitter.com/diniz_benedicto/status...,u:diniz_benedicto,c:@RogerioCorreia_ Rogério Correia . Estou len...,bolsonaro202304_n_00205.json,2023-04-29 19:20:14,tweet_00487107,convo_1652060609674969091,id_1608481812707250176,diniz_benedicto,https://twitter.com/diniz_benedicto/status/165...,@RogerioCorreia_ Rogério Correia . Estou lendo...,"Rogério Correia. Estou lendo ""Os Demônios Desc..."
17175,t017175,v:1547227306913153026,d:2023-04-29,url:https://twitter.com/LuccaSo44679209/status...,u:LuccaSo44679209,"c:@CiresCanisio Não , o Lula não da uma refina...",bolsonaro202304_n_00205.json,2023-04-29 19:19:18,tweet_00487120,convo_1652359568280678401,id_1547227306913153026,LuccaSo44679209,https://twitter.com/LuccaSo44679209/status/165...,"@CiresCanisio Não , o Lula não da uma refinari...","Não, o Lula não da uma refinaria atroca de pro..."
17176,t017176,v:1554492869825683457,d:2023-04-29,url:https://twitter.com/Andre19lll/status/1652...,u:Andre19lll,c:@eunaovoupararde @CarlosZarattini Os índices...,bolsonaro202304_n_00205.json,2023-04-29 19:19:14,tweet_00487123,convo_1652308555968532481,id_1554492869825683457,Andre19lll,https://twitter.com/Andre19lll/status/16523085...,@eunaovoupararde @CarlosZarattini Os índices d...,"Os índices de desemprego, PIB, inflação e qual..."
17177,t017177,v:1585200142440882179,d:2023-04-29,url:https://twitter.com/priscila19865/status/1...,u:priscila19865,c:@ValS265451870 @Guthbsb @marcia_miami Tá err...,bolsonaro202304_n_00206.json,2023-04-29 19:18:59,tweet_00487127,convo_1651881289497149440,id_1585200142440882179,priscila19865,https://twitter.com/priscila19865/status/16518...,@ValS265451870 @

### Creating the file `file_index.txt`

In [36]:
df_tweets_filtered[['text_id', 'conversation', 'date', 'text_url']].to_csv('file_index.txt', sep=' ', index=False, header=False, encoding='utf-8', lineterminator='\n')

### Creating the file `tweets.txt`

In [37]:
folder = 'tweets'
try:
    os.mkdir(folder)
    print(f'Folder {folder} created!')
except FileExistsError:
    print(f'Folder {folder} already exists')

Folder tweets already exists


Note: The parameters `doublequote=False` and `escapechar=' '` are required to avoid that the column content is doublequoted with '"' in sentences that use characters that need to be escaped such as double quote '"' itself - this causes a malformed response from TreeTagger.

In [38]:
df_tweets_filtered[['text_id', 'conversation', 'date', 'user', 'content']].to_csv(f'{folder}/tweets.txt', sep='|', index=False, header=False, encoding='utf-8', lineterminator='\n', doublequote=False, escapechar=' ')

## Tagging with TreeTagger

- On Visual Studio Code (VS Code), open the folder where your project is located with `Open Folder...`
- Open a WSL Ubuntu Terminal on VS Code
- **Important**: Activate the `my_env` Python environment by executing `source "$HOME"/my_env/bin/activate`
- Proceed as indicated

Purpose: Annotate the texts in `tweets/tweets.txt` with part-of-speech and lemma information.
- Input
    - `file_index.txt`
    - `tweets/tweets.txt`
- Output
    - `tweets/tagged.txt`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash treetagging.sh
--- treetagging t000000 / t018205 ---
        reading parameters ...
        tagging ...
         finished.
--- treetagging t000001 / t018205 ---
        reading parameters ...
        tagging ...
         finished.
--- treetagging t000002 / t018205 ---
        reading parameters ...
        tagging ...
         finished.
--- treetagging t000003 / t018205 ---
        reading parameters ...
        tagging ...
         finished.
<omitted>
```

## Processing `tokenstypes`

Purpose: Capture the content tokens (specific occurrences of words) and the content types (general concept of words) from `tweets/tagged.txt`.
- Input
    - `file_index.txt`
    - `tweets/tagged.txt`
- Output
    - `tweets/tokens.txt`
    - `tweets/types.txt`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash tokenstypes.sh
--- tokenstypes t000000 / 18206 ---
--- tokenstypes t000001 / 18206 ---
--- tokenstypes t000002 / 18206 ---
--- tokenstypes t000003 / 18206 ---
--- tokenstypes t000004 / 18206 ---
--- tokenstypes t000005 / 18206 ---
<omitted>
```

## Processing `toplemmas`

Purpose: Determine the 1.000 top lemmas. **Important**: This process requires manual inspection. Non-meaningful lemmas should be excluded by updating `stoplist.sed` and reiterating the processing.
- Input
    - `tweets/types.txt`
    - `stoplist.sed`: List of rules that allows the exclusion of a certain lemmas
- Output
    - `selectedwords` = `var_index.txt`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash toplemmas.sh
```

## Processing `sas`

Purpose: Prepare input data for processing in SAS.
- Input
    - `tweets/types.txt`
    - `selectedwords`
    - `file_index.txt`
- Output
    - `columns`
    - `sas/data.txt`
    - `sas/dates.txt`
    - `sas/wcount.txt`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash sas.sh
--- v000001 ---
--- v000002 ---
--- v000003 ---
--- v000004 ---
--- v000005 ---
<omitted>
--- v001000 ---
[nltk_data] Downloading package punkt to /home/eyamrog/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Word counts written to sas/wcount.txt
```

## Processing `datamatrix`

Purpose: Prepares input data for calculating the correlation matrix.
- Input
    - `file_index.txt`
    - `columns`
    - `selectedwords`
- Output
    - `file_ids.txt`
    - `data.csv`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash datamatrix.sh
--- v000001 ---
--- v000002 ---
--- v000003 ---
--- v000004 ---
--- v000005 ---
<omitted>
--- v001000 ---
--- data.csv ...---
```

## Processing `correlationmatrix`

Purpose: Calculates the correlation matrix.
- Input
    - `data.csv`
- Output
    - `correlation`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash correlationmatrix.sh
--- python correlation ... ---
```

## Processing `formats`

Purpose: Prepare input data for processing in SAS.
- Input
    - `data.csv`
    - `selectedwords`
- Output
    - `sas/corr.txt`
    - `sas/word_labels_format.sas`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash formats.sh
--- sas/sas/corr.txt ---
--- sas/word_labels_format.sas ---
```

## Processing the statistical procedures on SAS

- Log in to your [SAS OnDemand for Academics](https://welcome.oda.sas.com/) account
- Proceed as indicated in this [video tutorial](https://youtu.be/I3u9zD3jyOA?si=68uIKVc2iusGG2KY)

## Processing `examples`

Purpose: Extract examples for analysis.
- Input
    - `sas/output_"$project"/loadtable.html`
    - `sas/output_"$project"/"$project"_scores.tsv`
    - `sas/output_"$project"/"$project"_scores_only.tsv`
- Output
    - `examples/factors`
    - `example files`

```
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash examples.sh
6780
1246
698
123
--- examples f1pos ---
--- factor 1 pos # 000001 ---
tr: warning: an unescaped backslash at end of string is not portable
--- factor 1 pos # 000002 ---
tr: warning: an unescaped backslash at end of string is not portable
--- factor 1 pos # 000003 ---
tr: warning: an unescaped backslash at end of string is not portable
--- factor 1 pos # 000004 ---
tr: warning: an unescaped backslash at end of string is not portable
--- factor 1 pos # 000005 ---
tr: warning: an unescaped backslash at end of string is not portable
<ommitted>
```

## Results

Right-click on the link and choose `Open link in a new tab` to download the corresponding file.

- [CL_St1_Querem_Results.zip](https://pucsp-my.sharepoint.com/:u:/g/personal/ra00341729_pucsp_edu_br/ERbP8OEqscBJlh4l6s6_UFgBTUGtnR6PDI1NXZwVBh6Dyg?e=W8YXpq)